In [3]:
import numpy as np
import pandas as pd
import sklearn as skl
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime
import random

from scipy import stats

from tensorflow.contrib.layers import dropout

In [62]:
def standardize_numpy(array):
    
    #if np.linalg.norm(list) == 0:
    #    return list
    #else:
    normalized_array = stats.zscore(array)
    return normalized_array

with open('C:/Users/Robin Köster/Desktop/Studienarbeit ML/08_06_EINS/GANs/Energy_Data/2015_2017_hourly_DEATLU_load_PV_wind_forecast_noNA.csv', newline = '') as csvfile:
    datareader = csv.reader(csvfile, delimiter = ';')
    
   # C:\Users\Robin Köster\Desktop\Studienarbeit ML\Energy_data
    
    
    data_array = np.array(list(datareader))    
    
date_np_array = np.array([np.array(data_array[i]) for i in range(1,len(data_array))])

prices = np.array(list(map(lambda it: float(date_np_array[it,4]) , range(1,len(date_np_array))) ))

prices_standardized = standardize_numpy(prices)

load_forecast = standardize_numpy(np.array((\
                                        list(map(\
                                                 lambda it: float(date_np_array[it,1]) ,\
                                                 range(1,len(date_np_array))) ))) )

wind_forecast = standardize_numpy(np.array((\
                                        list(map(\
                                                 lambda it: float(date_np_array[it,3]) , \
                                                 range(1,len(date_np_array))) ))) )

pv_forecast = standardize_numpy(np.array(( \
                                      list(map( \
                                               lambda it: float(date_np_array[it,2]) , \
                                               range(1,len(date_np_array))) ))) )


date_array = np.array(list(map(lambda it: datetime.strptime(date_np_array[it,0],'%d.%m.%Y %H:%M') , \
                               range(1,len(date_np_array))) ))

day_of_week_array = standardize_numpy(np.array((\
                        list(map(lambda date: datetime.weekday(date), date_array)))) )

hour_array = np.array(list(map(lambda date: datetime.time(date).hour, date_array)))

# splitting the data into trainings set and validation set

training_ratio = 0.8

ind_split = np.round(training_ratio * len(prices)).astype(int)

prices_training = prices[:ind_split]
prices_validation = prices[ind_split:]

prices_standardized_training = prices[:ind_split]
prices_standardized_validation = prices[ind_split:]

day_of_week_training = day_of_week_array[:ind_split]
day_of_week_validation = day_of_week_array[ind_split:]

hour_array_training = hour_array[:ind_split]
hour_array_validation = hour_array[ind_split:]

load_forecast_training = load_forecast[:ind_split]
load_forecast_validation = load_forecast[ind_split:]

wind_forecast_training = wind_forecast[:ind_split]
wind_forecast_validation = wind_forecast[ind_split:]

pv_forecast_training = pv_forecast[:ind_split]
pv_forecast_validation = pv_forecast[ind_split:]

In [63]:
tf.reset_default_graph()

In [64]:
# network architecture: 
# one LSTM network, which gets the prices of the previous day as input
# one MLP, which gets just the day of the week as input

# hyperparameters
lrate = 1e-3
mb_size = 50                  # size of mini batches
time_series_past_length = 24  # number of hours in the "conditional" time series as input
time_series_past_dim = 1      # number of featers included in the past time series
conditions_length = 24
conditions_dim = 4            # number of conditional inputs which should be processed by the MLP
time_series_pred_length = 24  # number of hours in the predicted time series as output
time_series_pred_dim = 1
n_LSTM = 50                   # number of neurons in the LSTM layer (number of neurons in each "step" in the unrolled RNN)
n_DNN = 100                      # number of neurons in the hidden layer of the DNN

no_its = 100000

def normalize_list_numpy(list):
    
    if np.linalg.norm(list) == 0:
        prin(list)
        return list
    else:
        normalized_list = list / np.linalg.norm(list)
    return normalized_list

def next_batch(size):
    
    ind_vec =  np.where(hour_array_training==0)[0]   # indices of data points with hour 0
    ind_vec = list(ind_vec[:-2])                     # skip the last two elements
    
    random.seed( datetime.now() )
    
    ind_vec_shuffled = random.sample(ind_vec,len(ind_vec))

    mb_ind = ind_vec_shuffled[0:(size)]
    
    mb_prices_day1 = np.array(list(map(lambda mb_first_ind: prices_training[mb_first_ind:(mb_first_ind+time_series_past_length)], mb_ind)))   
    mb_prices_day2 = \
        np.array(list(map(lambda mb_first_ind: \
                           prices_training[(mb_first_ind+time_series_past_length):\
                                   (mb_first_ind+time_series_past_length+time_series_pred_length)], mb_ind)))
    
    # compute the weekday of the day, for which prices have to be predicted
    weekday = \
        np.array((list(map(lambda mb_first_ind: \
                          day_of_week_training[mb_first_ind+time_series_past_length]*np.ones(1), \
                          mb_ind))))
    
    load_forecast = \
        np.array((list(map(lambda mb_first_ind: \
                           load_forecast_training[(mb_first_ind+time_series_past_length):\
                                   (mb_first_ind+time_series_past_length+time_series_pred_length)], mb_ind))))
    
    pv_forecast = \
        np.array((list(map(lambda mb_first_ind: \
                           pv_forecast_training[(mb_first_ind+time_series_past_length):\
                                   (mb_first_ind+time_series_past_length+time_series_pred_length)], mb_ind))))

    wind_forecast = \
        np.array((list(map(lambda mb_first_ind: \
                           wind_forecast_training[(mb_first_ind+time_series_past_length):\
                                   (mb_first_ind+time_series_past_length+time_series_pred_length)], mb_ind))))
    
    #forecasts = np.concatenate((weekday, load_forecast, pv_forecast, wind_forecast), axis = 1)
    #forecasts = np.concatenate((load_forecast, pv_forecast, wind_forecast), axis = 1)
    forecasts = np.concatenate((load_forecast, wind_forecast, pv_forecast), axis = 1)    
    return mb_prices_day1, mb_prices_day2, forecasts


# input nodes for the time series of the previous day: x(t) as exogeneous variables
past_input_nodes = tf.placeholder(tf.float32, shape=[None, time_series_past_length, time_series_past_dim])

# input nodes for the conditional inputs for the day ahead
future_input_nodes = tf.placeholder(tf.float32, shape=[None, conditions_length * (conditions_dim-1)])

# nodes which the real prices may be assigned to during the training procedure
real_prices_nodes = tf.placeholder(tf.float32, shape =[None, time_series_pred_length])

# boolean whether its a training run and whether the dropout should be applied accordingly
is_training = tf.placeholder(tf.bool, shape = (), name = 'is_training')


##########   LSTM    #############################

# create the LSTM cell
lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(\
                                tf.contrib.rnn.BasicLSTMCell \
                                (num_units = n_LSTM), output_size = 1)

# include the cell in the graph
lstm_output, lstm_states = tf.nn.dynamic_rnn(\
                            lstm_cell, past_input_nodes, dtype = tf.float32)

##########    DNN    #############################

DNN_h1 = tf.contrib.layers.fully_connected(future_input_nodes, n_DNN)

lstm_output = tf.squeeze(lstm_output, axis =[2])
########## MERGE  ################################

merged_states = tf.concat((DNN_h1, lstm_output), axis = 1)

output = tf.contrib.layers.fully_connected(merged_states, 24)

In [65]:
#### training of the model #######################

import pylab

loss = tf.reduce_mean(tf.square(output - real_prices_nodes))

loss += reg_term

optimizer = tf.train.AdamOptimizer(learning_rate = lrate)

training_op = optimizer.minimize(loss)

saver = tf.train.Saver()

if not os.path.exists('out/'):
    os.makedirs('out/')
    
if not os.path.exists('cpkt_files/'):
    os.makedirs('cpkt_files/')

sess = tf.Session()
sess.run(tf.global_variables_initializer())

i = 0

for it in range(no_its):
    X_mb_day1, X_mb_day2, forecasts = next_batch(mb_size)
    
    X_mb_day1_shaped = X_mb_day1[:,:,np.newaxis]
    
    sess.run(training_op, feed_dict = {past_input_nodes: X_mb_day1_shaped, \
                                       real_prices_nodes: X_mb_day2, \
                                       future_input_nodes: forecasts, \
                                       is_training: True})
    
    if it % 1000 == 0:
        
        X_mb_day1, X_mb_day2, forecasts = next_batch(1)
    
        X_mb_day1_shaped = X_mb_day1[:,:,np.newaxis]
    
        mse, prediction = sess.run([loss, output], \
                                   feed_dict = {past_input_nodes: X_mb_day1_shaped,\
                                                real_prices_nodes: X_mb_day2,\
                                                future_input_nodes: forecasts,\
                                                is_training: False})
        print(it, "\tMSE:", mse)
        
        
        fig = plt.figure(figsize=(15,8))
        fig = plt.plot(np.ndarray.flatten(X_mb_day2[0]), label='Real prices')
        fig = plt.plot(np.ndarray.flatten(prediction[0]), label='Prediction')
        pylab.legend(loc='upper left')

        plt.savefig('out/{}.png'
                    .format(str(i).zfill(4)), bbox_inches='tight')
        i += 1
        fig = plt.gcf()
        plt.close(fig)
    
    if it % 5000 == 0:
        save_path = saver.save(sess, "cpkt_files/model.cpkt")
        
save_path = saver.save(sess, "cpkt_files/model_final.cpkt")

0 	MSE: 846.6774
1000 	MSE: 19.150343
2000 	MSE: 58.818806
3000 	MSE: 24.865534
4000 	MSE: 7.538599
5000 	MSE: 6.6076694
6000 	MSE: 4.7036824
7000 	MSE: 4.535567
8000 	MSE: 2.1887705
9000 	MSE: 9.453052
10000 	MSE: 41.466442
11000 	MSE: 5.8194656
12000 	MSE: 12.427228
13000 	MSE: 20.28901
14000 	MSE: 4.2289615
15000 	MSE: 16.530798
16000 	MSE: 6.154017
17000 	MSE: 3.8965092
18000 	MSE: 4.002008
19000 	MSE: 9.605946
20000 	MSE: 15.894612
21000 	MSE: 5.0219793
22000 	MSE: 11.810532
23000 	MSE: 12.330302
24000 	MSE: 9.66838
25000 	MSE: 3.7147903
26000 	MSE: 30.74169
27000 	MSE: 1.9697766
28000 	MSE: 5.6441016
29000 	MSE: 8.595242
30000 	MSE: 14.800968
31000 	MSE: 3.8327048
32000 	MSE: 9.745759
33000 	MSE: 1.6422586
34000 	MSE: 4.1132913
35000 	MSE: 2.7953281
36000 	MSE: 6.058678
37000 	MSE: 4.95092
38000 	MSE: 10.361024
39000 	MSE: 10.334334
40000 	MSE: 13.760017
41000 	MSE: 12.598068
42000 	MSE: 22.513266
43000 	MSE: 2.8158529
44000 	MSE: 5.4200234
45000 	MSE: 9.417458
46000 	MSE: 60.649

In [66]:
# performance on training set

saver = tf.train.Saver()

with tf.Session() as sess:

    saver.restore(sess, "cpkt_files/model_final.cpkt")

    
    ind_vec =  np.where(hour_array_training==0)[0]   # indices of data points with hour 0
    ind_vec = list(ind_vec[:-2])    # skip the last two weeks since they may be uncomplete
    
    
    prices_day1 = np.array(list(map(lambda first_ind: \
                                           prices_training[first_ind:(first_ind+time_series_past_length)], ind_vec)))
    
    prices_day2 = \
        np.array(list(map(lambda first_ind: \
                           prices_training[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    
    pv_forecast = \
        np.array(list(map(lambda first_ind: \
                           pv_forecast_training[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    wind_forecast = \
        np.array(list(map(lambda first_ind: \
                           wind_forecast_training[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    load_forecast = \
        np.array(list(map(lambda first_ind: \
                           load_forecast_training[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    
    weekday = \
        np.array(list(map(lambda first_ind: \
                          day_of_week_training[first_ind+time_series_past_length] * np.ones(conditions_length), \
                          ind_vec)))
    
    forecasts = np.concatenate((load_forecast, wind_forecast, pv_forecast), axis = 1)
    
    forecasts = forecasts[:,np.newaxis]
    
    prices_day1 = prices_day1[:,np.newaxis,:, np.newaxis]
    prices_day2 = prices_day2[:,np.newaxis,:]
    
    sMAPE = 0;
    
    for ind in range(prices_day1.shape[0]):
        
        predictions = sess.run([output], feed_dict = {past_input_nodes: prices_day1[ind], \
                                           real_prices_nodes: prices_day2[ind], \
                                           future_input_nodes: forecasts[ind], \
                                           is_training: False})
        

        predictions_array = np.array(predictions).flatten()
        real_prices = prices_day2[ind].flatten()
    
        #print(np.intersect1d(np.where(predictions_array==0),np.where(real_prices==0)).size)

        
        if  np.intersect1d(np.where(predictions_array==0),np.where(real_prices==0)).size == 0:
            sMAPE += 100 / (len(ind_vec) * len(real_prices)) * np.sum(\
                                 np.array(list(map(lambda y, y_hat: np.absolute(y-y_hat)*2/(np.absolute(y)+np.absolute(y_hat)), \
                                  real_prices, predictions_array ) )))
        
    print("\n sMAPE: ",sMAPE)
    

INFO:tensorflow:Restoring parameters from cpkt_files/model_final.cpkt

 sMAPE:  7.775767826994347


In [67]:
# performance on validation set

saver = tf.train.Saver()

with tf.Session() as sess:

    saver.restore(sess, "cpkt_files/model_final.cpkt")

    
    ind_vec =  np.where(hour_array_validation==0)[0]   # indices of data points with hour 0
    ind_vec = list(ind_vec[:-2])    # skip the last two weeks since they may be uncomplete
    
    
    prices_day1 = np.array(list(map(lambda first_ind: \
                                           prices_validation[first_ind:(first_ind+time_series_past_length)], ind_vec)))
    
    prices_day2 = \
        np.array(list(map(lambda first_ind: \
                           prices_validation[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    
    pv_forecast = \
        np.array(list(map(lambda first_ind: \
                           pv_forecast_validation[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    wind_forecast = \
        np.array(list(map(lambda first_ind: \
                           wind_forecast_validation[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    load_forecast = \
        np.array(list(map(lambda first_ind: \
                           load_forecast_validation[(first_ind+time_series_past_length):\
                                   (first_ind+time_series_past_length+time_series_pred_length)], ind_vec)))
    
    weekday = \
        np.array(list(map(lambda first_ind: \
                          day_of_week_validation[first_ind+time_series_past_length] * np.ones(conditions_length), \
                          ind_vec)))
    
    forecasts = np.concatenate((load_forecast, wind_forecast, pv_forecast), axis = 1)
    
    forecasts = forecasts[:,np.newaxis]
    
    prices_day1 = prices_day1[:,np.newaxis,:, np.newaxis]
    prices_day2 = prices_day2[:,np.newaxis,:]
    
    sMAPE = 0;
    
    for ind in range(prices_day1.shape[0]):
        
        predictions = sess.run([output], feed_dict = {past_input_nodes: prices_day1[ind], \
                                           real_prices_nodes: prices_day2[ind], \
                                           future_input_nodes: forecasts[ind], \
                                           is_training: False})
        

        predictions_array = np.array(predictions).flatten()
        real_prices = prices_day2[ind].flatten()
    
        #print(np.intersect1d(np.where(predictions_array==0),np.where(real_prices==0)).size)

        
        if  np.intersect1d(np.where(predictions_array==0),np.where(real_prices==0)).size == 0:
            sMAPE += 100 / (len(ind_vec) * len(real_prices)) * np.sum(\
                                 np.array(list(map(lambda y, y_hat: np.absolute(y-y_hat)*2/(np.absolute(y)+np.absolute(y_hat)), \
                                  real_prices, predictions_array ) )))
        
    print("\n sMAPE: ",sMAPE)

INFO:tensorflow:Restoring parameters from cpkt_files/model_final.cpkt

 sMAPE:  21.429276121074903
